# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2026

# Integer programming -- Knapsack problem example

# Week 5



# The Hiking Trip Problem
You're going on a day hike and can carry at most 15 kg in your backpack. You have 6 items to choose from:

| Item | Weight (kg) | Value/utility |
|------|-------------|-------------------|
| Water | 3 | 10 |
| Food | 4 | 9 |
| First-aid kit | 2 | 7 |
| Rain jacket | 2 | 6 |
| Camera | 3 | 5 |
| Book | 3 | 3 |

The goal is to maximize the utility of objects packed in your backpack.

We are going to use `PuLP` to solve this problem.

`PuLP` uses branch-and-bound internally, similar to what we discussed earlier, but with enhancements:
1. It solves the LP relaxation (allows fractional quantities) to get an upper bound
2. It intelligently chooses which variable to branch on using heuristics like:

Variables closest to 0.5 (most fractional)
Variables that appear in many constraints
Problem-specific strategies

3. It prunes branches when the LP relaxation can't beat the best integer solution found so far
4. Modern solvers add many optimizations:

Cutting planes - add extra constraints to tighten the LP relaxation
Heuristics - quickly find good (not necessarily optimal) integer solutions
Preprocessing - simplify the problem before solving
Parallel search - explore multiple branches simultaneously

In [47]:
from pulp import *

# Create problem
prob = LpProblem("Knapsack", LpMaximize)

# Items: (name, weight, value)
items = {
    'Water':      {"weight": 3, "utility": 10},
    'Food':       {"weight": 4, "utility":  9},
    'FirstAid':   {"weight": 2, "utility":  7},
    'RainJacket': {"weight": 2, "utility":  6},
    'Camera':     {"weight": 3, "utility":  5},
    'Book':       {"weight": 3, "utility":  3}
}


# Binary variables: take item or not
x = {name: LpVariable(name, cat='Binary') for name in items}
x


{'Water': Water,
 'Food': Food,
 'FirstAid': FirstAid,
 'RainJacket': RainJacket,
 'Camera': Camera,
 'Book': Book}

In [21]:
# we can get all the 'utility' values:
[ items[name]["utility"] * x[name] for name in items ]

[10*Water + 0,
 9*Food + 0,
 7*FirstAid + 0,
 6*RainJacket + 0,
 5*Camera + 0,
 3*Book + 0]

In [49]:
2*x["Water"]

2*Water + 0

In [22]:
# we need to create the objective function from these utility values.
# pulp has us do so by "adding" this vector to the problem using the += operator.

# Objective: maximize total utility
prob += lpSum(items[name]["utility"] * x[name] for name in items)

## or the following also works, but is slightly less efficient for a large number of items

# prob += lpSum( [items[name]["utility"] * x[name] for name in items] )



In [23]:

# and similarly, we "add" the constraint(s) to the problem

# Constraint: total weight <= 15
prob += lpSum(items[name]["weight"] * x[name] for name in items) <= 15

# again, we could give the vector as a list rather than a generator

# prob += lpSum( [items[name]["weight"] * x[name] for name in items] ) <= 15

In [51]:

# Create problem
prob = LpProblem("Knapsack", LpMaximize)

# Items: (name, weight, value)
items = {
    'Water':      {"weight": 3, "utility": 10},
    'Food':       {"weight": 4, "utility":  9},
    'FirstAid':   {"weight": 2, "utility":  7},
    'RainJacket': {"weight": 2, "utility":  6},
    'Camera':     {"weight": 3, "utility":  5},
    'Book':       {"weight": 3, "utility":  3}
}


# Binary variables: take item or not <-> 1 or 0
x = {name: LpVariable(name, cat='Binary') for name in items}

# Objective: maximize total utility
prob += lpSum(items[name]["utility"] * x[name] for name in items)

# Constraint: total weight <= 15
prob += lpSum(items[name]["weight"] * x[name] for name in items) <= 15

# and then we run the solver:
#prob.solve()

# this runs with less output noise...
prob.solve(PULP_CBC_CMD(msg=0))

# Print solution
print(f"Status: {LpStatus[prob.status]}")
print(f"Total utilty: {value(prob.objective)}")
print(f"Total weight: {sum(items[name]["weight"] * value(x[name]) for name in items)}")
print("\nItems to pack:")
for name in items:
    if value(x[name]) > 0:
        print(f"  {name:>10}: weight={items[name]["weight"]}, utility={items[name]["utility"]}")

Status: Optimal
Total utilty: 37.0
Total weight: 14.0

Items to pack:
       Water: weight=3, utility=10
        Food: weight=4, utility=9
    FirstAid: weight=2, utility=7
  RainJacket: weight=2, utility=6
      Camera: weight=3, utility=5


# Modified problem

Let's consider the same list of items, but now let's allow the packer to pack 0, 1, or 2 of each item.

So: rather than treating the variables as being `binary`, we'll treat the variables as integers with a bound `<= 2`.

So the code only changes when defining the *variables*.

In [50]:

# Create problem
prob = LpProblem("Knapsack", LpMaximize)

# Items: (name, weight, value)
items = {
    'Water':      {"weight": 3, "utility": 10},
    'Food':       {"weight": 4, "utility":  9},
    'FirstAid':   {"weight": 2, "utility":  7},
    'RainJacket': {"weight": 2, "utility":  6},
    'Camera':     {"weight": 3, "utility":  5},
    'Book':       {"weight": 3, "utility":  3}
}


# Integer variables: can take 0, 1, or 2 of each item
x = {name: LpVariable(name, lowBound=0, upBound=2, cat='Integer') 
     for name in items}

# Objective: maximize total utility
prob += lpSum(items[name]["utility"] * x[name] for name in items)

# Constraint: total weight <= 15
prob += lpSum(items[name]["weight"] * x[name] for name in items) <= 15

# and then we run the solver:
#prob.solve(msg=0)

prob.solve(PULP_CBC_CMD(msg=0))

# Print solution
print(f"Status: {LpStatus[prob.status]}")
print(f"Total utilty: {value(prob.objective)}")
print(f"Total weight: {sum(items[name]["weight"] * value(x[name]) for name in items)}")
print("\nItems to pack:")
for name in items:
    if value(x[name]) > 0:
        print(f"  {name:<10}: count={value(x[name]):2}, weight={items[name]["weight"]}, utility={items[name]["utility"]}")

Status: Optimal
Total utilty: 46.0
Total weight: 14.0

Items to pack:
  Water     : count=2.0, weight=3, utility=10
  FirstAid  : count=2.0, weight=2, utility=7
  RainJacket: count=2.0, weight=2, utility=6


One further tweak. Let's suppose that varying amounts of the items were available:

| Item | Weight (kg) | Value (usefulness) | # available |
|------|-------------|-------------------|--------------|
| Water | 3 | 10 | 2 |
| Food | 4 | 9 | 2 |
| First-aid kit | 2 | 7 | 1 |
| Rain jacket | 2 | 6 | 3 |
| Camera | 3 | 5 | 2|
| Book | 3 | 3 | 5 |


In [44]:

# Create problem
prob = LpProblem("Knapsack", LpMaximize)

# Items: a dictionary keyed on item type
# the values of the dictionary are themselves dictionaries, 
# each with "weight" in kg, "utility" or value and the  number of the item "avail"able
items = {
    'Water':      {"weight": 3, "utility": 10, "avail": 2},
    'Food':       {"weight": 4, "utility":  9, "avail": 2},
    'FirstAid':   {"weight": 2, "utility":  7, "avail": 1},
    'RainJacket': {"weight": 2, "utility":  6, "avail": 3},
    'Camera':     {"weight": 3, "utility":  5, "avail": 2},
    'Book':       {"weight": 3, "utility":  3, "avail": 5}
}


# Integer variables: can be between 0 and the number available for this item
x = {name: LpVariable(name, lowBound=0, upBound=items[name]["avail"], cat='Integer') 
     for name in items}

# Objective: maximize total utility
prob += lpSum(items[name]["utility"] * x[name] for name in items)

# Constraint: total weight <= 15
prob += lpSum(items[name]["weight"] * x[name] for name in items) <= 15

# and then we run the solver:
#prob.solve(msg=0)

prob.solve(PULP_CBC_CMD(msg=0))

# Print solution
print(f"Status: {LpStatus[prob.status]}")
print(f"Total utilty: {value(prob.objective)}")
print(f"Total weight: {sum(items[name]["weight"] * value(x[name]) for name in items)}")
print("\nItems to pack:")
for name in items:
    if value(x[name]) > 0:
        print(f"  {name:<10}: count={value(x[name]):2}, weight={items[name]["weight"]:5}, utility={items[name]["utility"]:5}")

Status: Optimal
Total utilty: 45.0
Total weight: 14.0

Items to pack:
  Water     : count=2.0, weight=    3, utility=   10
  FirstAid  : count=1.0, weight=    2, utility=    7
  RainJacket: count=3.0, weight=    2, utility=    6


Finally, we repeat the above problem, but now require there to be no more than 5 items in the backpack.


In [53]:
# Create problem
prob = LpProblem("Knapsack", LpMaximize)

# Items: a dictionary keyed on item type
# the values of the dictionary are themselves dictionaries, 
# each with "weight" in kg, "utility" or value and the  number of the item "avail"able
items = {
    'Water':      {"weight": 3, "utility": 10, "avail": 2},
    'Food':       {"weight": 4, "utility":  9, "avail": 2},
    'FirstAid':   {"weight": 2, "utility":  7, "avail": 1},
    'RainJacket': {"weight": 2, "utility":  6, "avail": 3},
    'Camera':     {"weight": 3, "utility":  5, "avail": 2},
    'Book':       {"weight": 3, "utility":  3, "avail": 5}
}


# Integer variables: can be between 0 and the number available for this item
x = {name: LpVariable(name, lowBound=0, upBound=items[name]["avail"], cat='Integer') 
     for name in items}

# Objective: maximize total utility
prob += lpSum(items[name]["utility"] * x[name] for name in items)

# Constraint: total weight <= 15
prob += lpSum(items[name]["weight"] * x[name] for name in items) <= 15

prob += lpSum(x[name] for name in items) <= 5

# and then we run the solver:
#prob.solve(msg=0)

prob.solve(PULP_CBC_CMD(msg=0))

# Print solution
print(f"Status: {LpStatus[prob.status]}")
print(f"Total utilty: {value(prob.objective)}")
print(f"Total weight: {sum(items[name]["weight"] * value(x[name]) for name in items)}")
print("\nItems to pack:")
for name in items:
    if value(x[name]) > 0:
        print(f"  {name:<10}: count={value(x[name]):2}, weight={items[name]["weight"]}, utility={items[name]["utility"]}")

Status: Optimal
Total utilty: 42.0
Total weight: 14.0

Items to pack:
  Water     : count=2.0, weight=3, utility=10
  Food      : count=1.0, weight=4, utility=9
  FirstAid  : count=1.0, weight=2, utility=7
  RainJacket: count=1.0, weight=2, utility=6


# Random knapsack problems

We can use `numpy` to generate *random* vectors and matrices.
Let's see how:

In [3]:
import numpy as np

# we can make a vector length 10 with random floats between 0 and 1 as follows:

np.random.rand(10)

array([0.80936086, 0.14142657, 0.02707956, 0.46146433, 0.40643622,
       0.97215423, 0.19410102, 0.75194957, 0.40502337, 0.16028397])

In [13]:
# or a 5 by 5 matrix
np.random.rand(5,5)

array([[0.87850764, 0.30133017, 0.43679015, 0.11714949, 0.37216147],
       [0.26321141, 0.77637687, 0.62497172, 0.26753159, 0.40985081],
       [0.13300946, 0.70757474, 0.17198717, 0.55059606, 0.07680364],
       [0.02098961, 0.77340991, 0.74317921, 0.41696958, 0.66286019],
       [0.53941942, 0.23918576, 0.66583804, 0.24070738, 0.02607131]])

In [11]:
# we can get random *integers* too

np.random.randint(0,10,size=(5,5))  # 5 by 5 matrix of random integers between 0 and 10

array([[4, 9, 4, 3, 9],
       [6, 0, 6, 8, 7],
       [6, 5, 6, 4, 4],
       [3, 9, 7, 6, 0],
       [8, 4, 0, 8, 8]])

Now, we want to create a large integer programming (knapsack) problem.

So: we make the following:

- objective function `c`
- upper bound matrix `A_ub`
- constraints `b_ub`

Since we want to find integer solutions, we'll arrange that the entries of `c`, `A_ub` and `b_ub` are all integers in some range.
And since we are imagining a knapsack-like problem, we'll even take all the coefficients to be non-negative integers.

In [93]:
import time
from scipy.optimize import linprog

# Problem size: 80 items, 5 knapsack constraints
n_items = 80
n_constraints = 5

# objective function
values = np.random.randint(10, 100, size=n_items)

weights = np.random.randint(1, 50, size=(n_constraints, n_items))
capacities = np.sum(weights, axis=1) * 0.4  # 40% of total capacity

# solve relaxed linear program using `scipy.optimize.linprog`

# objective function (remember, we are maximizing).
c = -values

# Inequality constraints: weights @ x <= capacities
A_ub = weights
b_ub = capacities

# Bounds: 0 <= x <= 1 (relaxed from binary)
bounds = [(0, 1) for _ in range(n_items)]

start = time.time()
result_relaxed = linprog(c, 
                         A_ub=A_ub, 
                         b_ub=b_ub, 
                         bounds=bounds, 
                         method='highs')
scipy_time = time.time() - start

print(f"Scipy (LP relaxation): {scipy_time:.4f} seconds")
print(f"Relaxed objective value: {-result_relaxed.fun:.2f}")
print(f"Fractional solutions: {np.sum((result_relaxed.x > 0.01) & (result_relaxed.x < 0.99))}")

values.shape,weights.shape, capacities.shape

Scipy (LP relaxation): 0.0048 seconds
Relaxed objective value: 2510.95
Fractional solutions: 5


((80,), (5, 80), (5,))

In [113]:

import pulp

prob = pulp.LpProblem("Knapsack", pulp.LpMaximize)
x = [pulp.LpVariable(f"x{i}", cat='Binary') for i in range(n_items)]

# Objective
prob += pulp.lpSum([values[i] * x[i] for i in range(n_items)])

# Constraints
for j in range(n_constraints):
    prob += pulp.lpSum([weights[j, i] * x[i] for i in range(n_items)]) <= capacities[j]

start = time.time()
prob.solve(pulp.PULP_CBC_CMD(msg=0))
pulp_time = time.time() - start

print(f"\nPuLP (Integer): {pulp_time:.4f} seconds")
print(f"Integer objective value: {pulp.value(prob.objective):.2f}")
print(f"Slowdown: {pulp_time/scipy_time:.1f}x slower")



PuLP (Integer): 0.0679 seconds
Integer objective value: 2491.00
Slowdown: 14.2x slower


In [114]:
# Let's automate, so we can change the number of items and the number of constraints

def gen_params(n_items,n_constraints):
    # objective function
    values = np.random.randint(10, 100, size=n_items)

    weights = np.random.randint(1, 50, size=(n_constraints, n_items))
    
    capacities = np.sum(weights, axis=1) * 0.4  # 40% of total capacity

    return { "values": values,
             "weights": weights,
             "capacities": capacities }

def relaxed(params):
    n_constraints,n_items = params["weights"].shape
    
    c = (-1)*params["values"]
    A_ub = params["weights"]
    b_ub = params["capacities"]
    
    # Bounds: 0 <= x <= 1 (relaxed from binary)
    bounds = [(0, 1) for _ in range(n_items)]

    start = time.time()
    result = linprog(c,
                     A_ub = A_ub,
                     b_ub = b_ub,
                     bounds=bounds)
    run_time = time.time() - start
    return { "time": run_time,
             "value": result.fun,
             "fractional_count": np.sum((result.x > 0.01) & (result.x < 0.99)) 
            }
    
def int_prog(params):
    n_constraints,n_items = params["weights"].shape
    
    prob = pulp.LpProblem("Knapsack", pulp.LpMaximize)
    x = [ pulp.LpVariable(f"x{i}", cat='Binary') for i in range(n_items) ]

    # Objective
    prob += pulp.lpSum([params["values"][i] * x[i] for i in range(n_items)])

    # Constraints
    for j in range(n_constraints):
        prob += pulp.lpSum([params["weights"][j, i] * x[i] 
                            for i in range(n_items)]) <= params["capacities"][j]

    start = time.time()
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    run_time = time.time() - start

    return { "time": run_time,
             "value": pulp.value(prob.objective)
           }

def compare(n_items,n_constraints):
    params = gen_params(n_items,n_constraints)

    rel_result = relaxed(params)
    ip_result = int_prog(params)

    print(f"relaxed objective value: {-rel_result["value"]}")
    print(f"fraction relaxed solutions: {rel_result["fractional_count"]}")
    print(f"relaxed time: {rel_result["time"]}")

    print()
    print(f"integer objective value: {ip_result["value"]}")
    print(f"integer time: {ip_result["time"]}")

    print()
    print(f"slowdown: {ip_result["time"]/rel_result["time"]:.2f} x slower")
          
             
    


In [115]:
compare(80,30)

relaxed objective value: 2582.619252043658
fraction relaxed solutions: 11
relaxed time: 0.007090568542480469

integer objective value: 2533.0
integer time: 1.277052402496338

slowdown: 180.11 x slower


In [117]:
compare(100,40)

relaxed objective value: 3140.1406406410442
fraction relaxed solutions: 12
relaxed time: 0.010352134704589844

integer objective value: 3088.0
integer time: 2.4912102222442627

slowdown: 240.65 x slower
